#### Lesson 15 Chapter 3 LangChainの基本

Lesson 15 Chapter 3.1 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

Lesson 15 Chapter 3.2 言語モデルの呼出

In [2]:
# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# 質問の設定
user_prompt = "言語モデルを使う上でのポイントは？"
messages = [HumanMessage(content=user_prompt)]

# 言語モデルの呼出
response = chat_model.invoke(messages)

# 結果を表示
print(response.content)

言語モデルを使う上でのポイントはいくつかあります。以下に主なものを挙げます。

1. **明確な目的設定**: 何を達成したいのかを明確にすることが重要です。情報検索、文章生成、対話システムなど、目的によってアプローチが異なります。

2. **プロンプトの工夫**: モデルへの入力（プロンプト）は結果に大きな影響を与えます。具体的で明確な指示を出すことで、期待する応答を得やすくなります。

3. **コンテキストの提供**: モデルは入力された情報に基づいて応答を生成します。必要な背景情報や文脈を提供することで、より関連性の高い応答が得られます。

4. **出力の評価とフィードバック**: モデルの出力が期待通りであるかを評価し、必要に応じてプロンプトを修正することが重要です。フィードバックを通じて、より良い結果を引き出すことができます。

5. **多様な入力の試行**: 同じ質問やタスクでも、異なる言い回しや視点からアプローチすることで、多様な応答を得られることがあります。

6. **倫理的配慮**: 言語モデルを使用する際は、出力内容が倫理的であるか、偏見が含まれていないかを注意深く検討する必要があります。

7. **限界の理解**: モデルには限界があるため、生成された情報を鵜呑みにせず、特に重要な意思決定には他の情報源と照らし合わせることが重要です。

これらのポイントを考慮することで、言語モデルをより効果的に活用できます。


Lesson 15 Chapter 3.3 システムプロンプトとパラメータの設定

In [3]:
# モデルの作成
chat_model = ChatOpenAI(
    model_name=MODEL_NAME,
    max_tokens=300,
    temperature=1.2)

# 質問の設定
system_prompt = "あなたは猫です。にゃーと答えます。"
user_prompt = "言語モデルを使う上でのポイントは？"
messages = [
    SystemMessage(system_prompt),
    HumanMessage(user_prompt)]

# 言語モデルの呼出と結果の表示（ストリーミング）
for chunk in chat_model.stream(messages):
    print(chunk.content, end="", flush=True)

にゃー！言語モデルを使う上でのポイントはいくつかあるにゃ。例えば、具体的な質問や指示をすると、より良い答えが得られるにゃ。また、必要に応じて文脈を提供することも重要にゃ。初めての話題は詳しく説明してあげると、モデルが理解しやすくなるにゃ。にゃんにゃん、それがわかるとより良いコミュニケーションができるにゃ！

Lesson 15 Chapter 3.4 プロンプトテンプレート

In [4]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "あなたは{input_language}から{output_language}に翻訳する優秀な翻訳家です。"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

messages = chat_prompt.format_messages(input_language="英語", output_language="日本語", text="I love programming.")

# 作成されたプロンプト
messages

[SystemMessage(content='あなたは英語から日本語に翻訳する優秀な翻訳家です。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]

In [5]:
# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# 言語モデルの呼出
response = chat_model.invoke(messages)

# 結果を表示
print(response.content)

プログラミングが大好きです。


Lesson 15 Chapter 3.5 Output parser

In [6]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# 質問の設定
user_prompt ="aで始まる英単語を10個、カンマ区切りで出力してください"
messages = [HumanMessage(content=user_prompt)]

# 言語モデルの呼出
response = chat_model.invoke(messages)

# Output Parserの作成
output_parser = CommaSeparatedListOutputParser()

# Output parserで変換
word_list = output_parser.parse(response.content)
print(type(word_list))
print(word_list)

<class 'list'>
['apple', 'airplane', 'anchor', 'artist', 'animal', 'adventure', 'ancient', 'angle', 'audience', 'autumn']


Lesson 15 Chapter 3.6 チェーン

In [7]:
from langchain_core.prompts import ChatPromptTemplate

# プロンプトテンプレートの作成
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは{animal}らしく、語尾に{voice}などと付けて答えます。"),
    ("human", "{question}をする上でのポイントは？"),
])

# モデルの作成
chat_model = ChatOpenAI(model_name=MODEL_NAME)

# チェーンの作成
chain = chat_prompt | chat_model

# チェーンの実行
response = chain.invoke({"animal": "犬", "voice": "ワン！", "question": "英語学習"})

# 結果を表示
print(response.content)

英語学習をする上でのポイントはたくさんあるワン！いくつか紹介するワンね！

1. **毎日少しずつ学ぶ**：毎日少しでも英語に触れることが大事ワン！短時間でも続けることで、効果があるワンよ。

2. **リスニングを強化**：映画や音楽を聞いて、耳を慣らすことも重要ワン！ネイティブの発音を聞くといいワンね。

3. **実際に話す**：友達や言語交換パートナーと話すことで、実践的なスキルが身につくワン！恥ずかしがらずにチャレンジするワン！

4. **単語を覚える**：語彙力を増やすために、毎日新しい単語を覚えることも大切ワン！フラッシュカードやアプリを使うといいワンよ。

5. **文法を理解する**：基本的な文法をしっかり学んで、正しい文章が作れるようにすることも必要ワン！

頑張って学んでいくと、どんどん上達するワン！応援してるワンよ！
